In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json

from skimage.io import imread, imsave
from skimage.measure import find_contours
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage.morphology import binary_erosion
from skimage.draw import polygon
from skimage.transform import warp_polar
from skimage.transform import warp, EuclideanTransform

from scipy.ndimage import distance_transform_edt
from scipy.optimize import fmin, least_squares
from scipy.signal import savgol_filter
from scipy.io import savemat

In [3]:
folder = '/mnt/ff9e5a34-3696-46e4-8fa8-0171539135be/Tweez scope'
exp_date = '2023_11_28'
path = os.path.join(folder, exp_date)
folder_masks = 'contour_masks'
fname = '2023_11_28_10x_1.0x_pAAA_TiTweez_Pos1.ome.tif'
fname_mask = 'mask_' + fname

path_im = os.path.join(path, fname)
path_masks = os.path.join(path, folder_masks, fname_mask)

In [5]:
im_all = imread(path_im)
im_ph = im_all[:,:,:,3]
im_ph = im_ph.astype(float)

In [6]:
nt,nx,ny = im_ph.shape

In [7]:
im_ph.shape

(216, 1024, 1024)

In [11]:
(nt,) + (nx,ny)

(216, 1024, 1024)

In [ ]:
date = '2023_11_15'
pos = [0,1,3]
rads = 100
524, 480
505,520

510, 510

In [10]:
metadata = {
    'Tweez scope': {
        '2023_11_15': {
            #'0': {'cx': 524, 'cy': 480, 'radius': 100}, #originally taken the other way around, (x, y) instead of (row, col)
            '0': {'cx': 524, 'cy': 480, 'radius': 100, 'radj': 50},
            '1': {'cx': 505, 'cy': 520, 'radius': 100, 'radj': 50},
            '3': {'cx': 510, 'cy': 510, 'radius': 100, 'radj': 50},
            # More positions
        },
        # More dates
    },
    'Ti scope': {
        '2023_11_15': {
            '0': {'cx': 600, 'cy': 600, 'radius': 100, 'radj': 50},
            '1': {'cx': 600, 'cy': 600, 'radius': 100, 'radj': 50},
            '2': {'cx': 600, 'cy': 600, 'radius': 100, 'radj': 50},
            '3': {'cx': 600, 'cy': 600, 'radius': 100, 'radj': 50},
            '6': {'cx': 625, 'cy': 590, 'radius': 110, 'radj': 50},
            '7': {'cx': 600, 'cy': 580, 'radius': 50, 'radj': 35},
            '9': {'cx': 605, 'cy': 590, 'radius': 100, 'radj': 50},
            '17': {'cx': 610, 'cy': 590, 'radius': 100, 'radj': 35},
            '19': {'cx': 965, 'cy': 750, 'radius': 100, 'radj': 35},
            # More positions
        },
        # More dates
    }
}

In [9]:
parsed = json.loads(metadata)
print(json.dumps(parsed, indent=4))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [11]:
with open("metadata.json", "w") as outfile: 
    json.dump(metadata, outfile, indent=4)

In [23]:
def get_params_for_date(microscope, date, metadata):
    """Retrieve all parameters for a given microscope and date."""
    try:
        return metadata[microscope][date]
    except KeyError:
        print("Data not found for the given microscope and date.")

In [27]:
# Example usage
scope_name = 'Tweez scope'
exp_date = '2023_11_15'
colonies = get_params_for_date(scope_name, exp_date, metadata)

result = get_params_for_date('Tweez scope', '2023_11_15', metadata)
result

{'0': {'cx': 524, 'cy': 480, 'radius': 100},
 '1': {'cx': 505, 'cy': 520, 'radius': 100},
 '3': {'cx': 510, 'cy': 510, 'radius': 100}}

In [18]:
list(result.keys())

['0', '1', '3']

In [25]:
result

{'0': {'cx': 524, 'cy': 480, 'radius': 100},
 '1': {'cx': 505, 'cy': 520, 'radius': 100},
 '3': {'cx': 510, 'cy': 510, 'radius': 100}}

In [29]:
for col in result.keys():
    cx = metadata[scope_name][exp_date][col]['cx']
    cy = metadata[scope_name][exp_date][col]['cy']
    print(cx, cy)

524 480
505 520
510 510


NameError: name 'col' is not defined

In [30]:
str(0)

'0'

In [31]:
def make_video(images_folder, video_name):
    """
    Makes a video from a sequence of png images.
    
    Parameters:
    - images_folder (string): Folder that contains the png files.
    - video_name (string): Date of the experimental data to be analyzed.

    Returns:
    - 
    """
    images = [img for img in os.listdir(images_folder) if img.endswith(".png")]
    images.sort()
    frame = cv2.imread(os.path.join(images_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, 7, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(images_folder, image)))

    video.release()

In [34]:
folder = '/mnt/ff9e5a34-3696-46e4-8fa8-0171539135be'
scope_name = 'Tweez scope'
path_scope = os.path.join(folder, scope_name)
exp_date = '2023_11_15'
path = os.path.join(path_scope, exp_date)
folder_masks = 'contour_masks'
pos = 0

In [37]:
make_video(os.path.join(path, folder_masks,f"temp_pos{pos}"), 
           os.path.join(path, folder_masks,f"contour_pos{pos}.avi"))

In [ ]:
images = [img for img in os.listdir(images_folder) if img.endswith(".png")]